## Django

### Django MVT

![](https://processon.com/chart_image/5c1ca66ce4b095ccfeea964d.png)

### WSGI

WSGI(Web Service Gateway Interface)定义了WEB服务器与Python WEB应用程序标准接口，提高了WEB 应用在不同WEB服务的可移植性。

WSGI接口包含两侧：服务器（或者网关）侧和应用框架侧。服务器调用应用程序侧的callable对象。

一个callable对象可以是一个函数、方法、类或者有__call__方法的对象实例。


#### Application

```python

def simple_app(environ, start_response):
    """Simple possible application object"""
    status = '200 OK'
    response_headers = [('Content-Type', 'text/plain')]
    start_respnse(status, respone_headers)
    return ['Hello World!\n']


class AppClass(object):
    """Produce the same object, but using a class"""
    def __init__(self, environ, start_response):
        self.environ = environ
        self.start = start_response

    def __iter__(self):
        status = '200 OK'
        respones_headers = [('Content-Type', 'text/plain')]
        self.start(status, response_headers)
        yield 'Hello World!\n'
```

#### Web Service

```python

def run_with_cgi(application):
    environ = dict(os.environ.items())
    environ['wsgi.input'] = sys.stdin
    environ['wsgi.errors'] = sys.stderr
    environ['wsgi.version'] = (1, 0)
    environ['wsgi.multithread'] = True
    environ['wsgi.multiprocess'] = True
    environ['wsgi.run_once'] = True
    environ['wgi.url_scheme'] = 'http'

    headers_set = []
    headers_sent = []
    
    def write(data):
        if not headers_set:
            raise AssertionError('write() before start_respones()')
        elif not headers_sent:
            # Before the first outpu, send the stored headers
            status, respone_headers = headers_sent[:] = headers_set
            sys.stdout.write('Status: %s\r\n' % status)
            for header in respone_headers:
                sys.stdout.write('%s: %s\rn\n' % header)
            sys.stdout.write('\r\n')

        sys.stdout.write(data)
        sys.stdout.flush()

    def start_response(status, respone_headers):
        headers_set[:] = [status, respone_headers]
        return write

    result = application(environ, start_response)
    try:
        for data in resutl:
            if data:
                write(data)     # 直到body出现，才发送status, headers数据

        if not headers_sent:
            write('')   # 如果body为空，发送status, headers数据
    finally:
        if hasattr(result, 'close'):
            result.close()
```

### Django Middelware

* *https://github.com/django/deps/blob/master/final/0005-improved-middleware.rst*
* *https://docs.djangoproject.com/en/1.10/topics/http/middleware/*


```python
class MiddlewareMixin:
    def __init__(self, get_response=None):
        self.get_response = get_response
        super().__init__()

    def __call__(self, request):
        response = None
        if hasattr(self, 'process_request'):
            response = self.process_request(request)
        response = response or self.get_response(request)
        if hasattr(self, 'process_response'):
            response = self.process_response(request, response)
        return response
```

```python
from django.utils.depreaction import MiddlewareMixin


class ExampleMiddleware(MiddlewareMixin):
    def process_request(self, request):
        return None

    def process_exceptions(self, request, exception):
        return None  # or return HttpResponse()

    def process_response(self, request, response):
        return response
```

#### Load middleware

```python
class WSGIHandler(base.BaseHandler):
    request_class = WSGIRequest

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.load_middleware()
```

```python

class BaseHandler:
    _view_middleware = None
    _template_response_middleware = None
    _exception_middleware = None
    _middleware_chain = None

    def load_middleware(self):
        """
        Populate middleware lists from settings.MIDDLEWARE.

        Must be called after the environment is fixed (see __call__ in subclasses).
        """
        self._view_middleware = []
        self._template_response_middleware = []
        self._exception_middleware = []

        handler = convert_exception_to_response(self._get_response)
        for middleware_path in reversed(settings.MIDDLEWARE):
            middleware = import_string(middleware_path)
            try:
                mw_instance = middleware(handler)
            except MiddlewareNotUsed as exc:
                if settings.DEBUG:
                    if str(exc):
                        logger.debug('MiddlewareNotUsed(%r): %s', middleware_path, exc)
                    else:
                        logger.debug('MiddlewareNotUsed: %r', middleware_path)
                continue

            if mw_instance is None:
                raise ImproperlyConfigured(
                    'Middleware factory %s returned None.' % middleware_path
                )

            if hasattr(mw_instance, 'process_view'):
                self._view_middleware.insert(0, mw_instance.process_view)
            if hasattr(mw_instance, 'process_template_response'):
                self._template_response_middleware.append(mw_instance.process_template_response)
            if hasattr(mw_instance, 'process_exception'):
                self._exception_middleware.append(mw_instance.process_exception)

            handler = convert_exception_to_response(mw_instance)

        # We only assign to this when initialization is complete as it is used
        # as a flag for initialization being complete.
        self._middleware_chain = handler
```

#### Run Middleware

```python
class WSGIHandler(base.BaseHandler):
    request_class = WSGIRequest

    def __call__(self, environ, start_response):
        set_script_prefix(get_script_name(environ))
        signals.request_started.send(sender=self.__class__, environ=environ)
        request = self.request_class(environ)
        response = self.get_response(request)
```

```python

class BaseHandler:
    _view_middleware = None
    _template_response_middleware = None
    _exception_middleware = None
    _middleware_chain = None

    def get_response(self, request):
        """Return an HttpResponse object for the given HttpRequest."""
        # Setup default url resolver for this thread
        set_urlconf(settings.ROOT_URLCONF)
        response = self._middleware_chain(request)
        response._closable_objects.append(request)
        if response.status_code >= 400:
            log_response(
                '%s: %s', response.reason_phrase, request.path,
                response=response,
                request=request,
            )
        return response

```

### Django Signal

信号实现了一个复杂系统中子系统之间的解耦，一个子系统的状态发生改变时，通过信号同步（或通知）其他依赖于该系统的系统更新状态，实现了状态的一致性。

#### Theory

##### Create Signal

```python
class Signal:
    """
    Base class for all signals

    Internal attributes:

        receivers
            { receiverkey (id) : weakref(receiver) }
    """
    def __init__(self, providing_args=None, use_caching=False):
        """
        Create a new signal.

        providing_args
            A list of the arguments this signal can pass along in a send() call.
        """
        self.receivers = []
        if providing_args is None:
            providing_args = []
        self.providing_args = set(providing_args)
        self.lock = threading.Lock()
        self.use_caching = use_caching
        # For convenience we create empty caches even if they are not used.
        # A note about caching: if use_caching is defined, then for each
        # distinct sender we cache the receivers that sender has in
        # 'sender_receivers_cache'. The cache is cleaned when .connect() or
        # .disconnect() is called and populated on send().
        self.sender_receivers_cache = weakref.WeakKeyDictionary() if use_caching else {}
        self._dead_receivers = False
```

> lock 保证线程安全

> 为了防止调用被释放了对象，信号内部保持对receiver调用对象的弱引用，每次发送信号之前，检查该对象是否存在，如果不存在，则标记_dead_receivers为True,等待清楚。

##### Connect Signal

In [10]:

class Signal():

    def connect(self, receiver, sender=None, weak=True, dispatch_uid=None):
        """
        Connect receiver to sender for signal.

        Arguments:

            receiver
                A function or an instance method which is to receive signals.
                Receivers must be hashable objects.

                If weak is True, then receiver must be weak referenceable.

                Receivers must be able to accept keyword arguments.

                If a receiver is connected with a dispatch_uid argument, it
                will not be added if another receiver was already connected
                with that dispatch_uid.

            sender
                The sender to which the receiver should respond. Must either be
                a Python object, or None to receive events from any sender.

            weak
                Whether to use weak references to the receiver. By default, the
                module will attempt to use weak references to the receiver
                objects. If this parameter is false, then strong references will
                be used.

            dispatch_uid
                An identifier used to uniquely identify a particular instance of
                a receiver. This will usually be a string, though it may be
                anything hashable.
        """
        from django.conf import settings

        # If DEBUG is on, check that we got a good receiver
        if settings.configured and settings.DEBUG:
            assert callable(receiver), "Signal receivers must be callable."

            # Check for **kwargs
            if not func_accepts_kwargs(receiver):
                raise ValueError("Signal receivers must accept keyword arguments (**kwargs).")

        if dispatch_uid:
            lookup_key = (dispatch_uid, _make_id(sender))
        else:
            lookup_key = (_make_id(receiver), _make_id(sender))

        if weak:
            ref = weakref.ref
            receiver_object = receiver
            # Check for bound methods
            if hasattr(receiver, '__self__') and hasattr(receiver, '__func__'):
                ref = weakref.WeakMethod
                receiver_object = receiver.__self__
            receiver = ref(receiver)
            weakref.finalize(receiver_object, self._remove_receiver)

        with self.lock:
            self._clear_dead_receivers()
            if not any(r_key == lookup_key for r_key, _ in self.receivers):
                self.receivers.append((lookup_key, receiver))
            self.sender_receivers_cache.clear()

> 当weak参数为True时，signal内部维护的是对象的弱引用，当对象的引用计数为0，被GC释放时，`_remove_receiver`函数被回掉， `_dead_receivers`变量设置为True，

##### Send Signal

In [12]:
class Signal():
    def send(self, sender, **named):
        """
        Send signal from sender to all connected receivers.

        If any receiver raises an error, the error propagates back through send,
        terminating the dispatch loop. So it's possible that all receivers
        won't be called if an error is raised.

        Arguments:

            sender
                The sender of the signal. Either a specific object or None.

            named
                Named arguments which will be passed to receivers.

        Return a list of tuple pairs [(receiver, response), ... ].
        """
        if not self.receivers or self.sender_receivers_cache.get(sender) is NO_RECEIVERS:
            return []

        return [
            (receiver, receiver(signal=self, sender=sender, **named))
            for receiver in self._live_receivers(sender)
        ]


##### Disconnect Signal

In [14]:
class Signal():
    def disconnect(self, receiver=None, sender=None, dispatch_uid=None):
        """
        Disconnect receiver from sender for signal.

        If weak references are used, disconnect need not be called. The receiver
        will be removed from dispatch automatically.

        Arguments:

            receiver
                The registered receiver to disconnect. May be none if
                dispatch_uid is specified.

            sender
                The registered sender to disconnect

            dispatch_uid
                the unique identifier of the receiver to disconnect
        """
        if dispatch_uid:
            lookup_key = (dispatch_uid, _make_id(sender))
        else:
            lookup_key = (_make_id(receiver), _make_id(sender))

        disconnected = False
        with self.lock:
            self._clear_dead_receivers()
            for index in range(len(self.receivers)):
                (r_key, _) = self.receivers[index]
                if r_key == lookup_key:
                    disconnected = True
                    del self.receivers[index]
                    break
            self.sender_receivers_cache.clear()
        return disconnected

**观察者设计模式**

定义对象间一对多的依赖关系，当一个对象的状体发生改变时，所有依赖于它的对象都得到通知并被自动更新。

将一系统分割成一系列相互协作的类有一个副作用：需要维护对象间的一致性。我们不希望为了维持一致性而使各类聚合，这样就降低了可重用性。

observer模式描述了如何建立这种关系。这一模式的关键对象是目标（subject）和观察者（observer）.一个目标可以有任意数量的观察者。一旦目标的状态发生改变，所有的观察者都得到通知。作为对这个通知的响应，每个观察者都将查询目标以使其状态与目标的状态同步。

这种交互也成为发布-订阅（publish-subscribe）.目标是通知的发布者，可以有任意数目的观察者订阅并接收通知。

### Django Cache

*https://mengyangyang.org/2017/04/06/django-cache/*

每次用户请求一个页面，Web服务器都要进行很多计算，查询数据库，合成模板，处理业务逻辑等，再将页面返回给用户．后续的相同资源的请求，服务器都需要重复这些计算．

django提供了缓存机制，每次将资源的响应的副本存储指定的位置，下次用户再发起相同的请求时，服务器不再需要进行类似的计算，直接将上次响应的副本返回给用户．这样既减少服务器的负载，又降低用户的请求时延，提高了用户体验．

![](http://processon.com/chart_image/5a28e148e4b0dce08036fe4e.png)

#### CacheHandler


```python
class CacheHandler:
    """
    A Cache Handler to manage access to Cache instances.

    Ensure only one instance of each alias exists per thread.
    """
    def __init__(self):
        self._caches = local()

    def __getitem__(self, alias):
        try:
            return self._caches.caches[alias]
        except AttributeError:
            self._caches.caches = {}
        except KeyError:
            pass

        if alias not in settings.CACHES:
            raise InvalidCacheBackendError(
                "Could not find config for '%s' in settings.CACHES" % alias
            )

        cache = _create_cache(alias)
        self._caches.caches[alias] = cache
        return cache

    def all(self):
        return getattr(self._caches, 'caches', {}).values()


caches = CacheHandler()
```

#### Backends

采用模板方法的设计模式，通过抽象基类BaseCache声明了一套缓存操作接口，而将接口实现延迟到具体的子类中。

BaseCache 是一个抽象类，定义了缓存通用的操作接口和参数默认值．

其中重要的参数：

    key 生成机制
    timeout 超时时间
    max_entries 最大条目数
    cull_frequency 更新频率


#### Cache Middleware

缓存中间件的工作原理（参考实现代码）：

* 只有状态码为200的，方法为HEAD,GET请求的响应被缓存
* 检测缓存中是否已缓存该请求的响应对象
* 如果命中，返回原始响应对象的一个浅拷贝（shallow copy）
* 如果未命中，继续处理view函数
* 根据请求的header决定是否需要缓存
* 设置响应的ETag, Last-Modified, Expires, Cache-Control HTTP header.


> 在response阶段，中间件的处理顺序是bottom-top, UPdateCacheMiddleware必须最后被执行，因此放在靠前的位置．在request阶段，中间件的处理顺序是top-bottom, FetchFromCacheMiddleware必须最后被执行，因此放在靠后的位置．

```python

class FetchFromCacheMiddleware(MiddlewareMixin):
    
    def process_request(self, request):
      """
      Checks whether the page is already cached and returns the cached
      version if available.
      """
        # 只缓存HEAD, GET的请求
        if request.method not in ('GET', 'HEAD'):
          request._cache_update_cache = False
          return None  # Don't bother checking the cache.

# 获取GET方法的cache_key,如果不存在，则设置_cache_update_cache标志位为True，需要更新缓存
        # try and get the cached GET response
      cache_key = get_cache_key(request, self.key_prefix, 'GET', cache=self.cache)
      if cache_key is None:
          request._cache_update_cache = True
          return None  # No cache information available, need to rebuild.
      response = self.cache.get(cache_key)
        # 如果cache为命中，而且请求的方法为HEAD,则获取请求方法为HEAD的cache_key
        # if it wasn't found and we are looking for a HEAD, try looking just for that
      if response is None and request.method == 'HEAD':
          cache_key = get_cache_key(request, self.key_prefix, 'HEAD', cache=self.cache)
          response = self.cache.get(cache_key)

# 缓存都未命中，设置_cache_update_cache标志位为True,调用view函数，并更新缓存
        if response is None:
          request._cache_update_cache = True
          return None  # No cache information available, need to rebuild.

# 缓存命中，设置_cache_update_cahe标志为False, 不调用view函数，不更新缓存
        # hit, return cached response
      request._cache_update_cache = False
      return response
```

```python

class UpdateCacheMiddleware(MiddlewareMixin):

   def _should_update_cache(self, request, response):
      return hasattr(request, '_cache_update_cache') and request._cache_update_cache

  def process_response(self, request, response):
      """Sets the cache, if needed."""
        # 不缓存，直接返回Response
        if not self._should_update_cache(request, response):
          # We don't need to update the cache, just return.
          return response

# 如果是流数据或状态码不为200, 不缓存
        if response.streaming or response.status_code != 200:
          return response

# 如果是私有数据，不缓存
        # Don't cache responses that set a user-specific (and maybe security
      # sensitive) cookie in response to a cookie-less request.
      if not request.COOKIES and response.cookies and has_vary_header(response, 'Cookie'):
          return response

      # Try to get the timeout from the "max-age" section of the "Cache-
      # Control" header before reverting to using the default cache_timeout
      # length.
      timeout = get_max_age(response)
      if timeout is None:
          timeout = self.cache_timeout
      elif timeout == 0:
          # max-age was set to 0, don't bother caching.
          return response
      
        # 设置缓存的HTTP头部信息
        patch_response_headers(response, timeout)
        # 缓存响应
        if timeout:
          cache_key = learn_cache_key(request, response, timeout, self.key_prefix, cache=self.cache)
          if hasattr(response, 'render') and callable(response.render):
              response.add_post_render_callback(
                  lambda r: self.cache.set(cache_key, r, timeout)
              )
          else:
              self.cache.set(cache_key, response, timeout)
      return response
```

![](http://processon.com/chart_image/5a556a1ae4b0abe85d497768.png)

### Django Auth

```py
MIDDLEWARE = [
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
]
```

Session中间件`SessionMiddleware`用于加载和回写session信息。

Auth中间件`AuthenticationMiddleware`用于加载`user`对象（延迟加载），并作为`request`的属性保存起来。

#### Django Session

**Middleware**

`process_request`加载session，保存在`request.session`对象里， `process_response`检查是否更新，如果需要，则更新cookie.

```py
class SessionBase:
    """
    Base class for all Session classes.
    """
    TEST_COOKIE_NAME = 'testcookie'
    TEST_COOKIE_VALUE = 'worked'

    __not_given = object()

    def __init__(self, session_key=None):
        self._session_key = session_key
        self.accessed = False
        self.modified = False
        self.serializer = import_string(settings.SESSION_SERIALIZER)

    def __contains__(self, key):
        return key in self._session

    def __getitem__(self, key):
        return self._session[key]

    def __setitem__(self, key, value):
        self._session[key] = value
        self.modified = True

    def __delitem__(self, key):
        del self._session[key]
        self.modified = True
```

> 重载`_getitem__`, `__setitem__`, `__delitem__`方法，记录session的更新。

```py
class SessionMiddleware(MiddlewareMixin):
    def __init__(self, get_response=None):
        self.get_response = get_response
        engine = import_module(settings.SESSION_ENGINE)
        self.SessionStore = engine.SessionStore

    def process_request(self, request):
        session_key = request.COOKIES.get(settings.SESSION_COOKIE_NAME)
        request.session = self.SessionStore(session_key)

    def process_response(self, request, response):
        """
        If request.session was modified, or if the configuration is to save the
        session every time, save the changes and set a session cookie or delete
        the session cookie if the session has been emptied.
        """
        try:
            accessed = request.session.accessed
            modified = request.session.modified
            empty = request.session.is_empty()
        except AttributeError:
            pass
        else:
            # First check if we need to delete this cookie.
            # The session should be deleted only if the session is entirely empty
            if settings.SESSION_COOKIE_NAME in request.COOKIES and empty:
                response.delete_cookie(
                    settings.SESSION_COOKIE_NAME,
                    path=settings.SESSION_COOKIE_PATH,
                    domain=settings.SESSION_COOKIE_DOMAIN,
                )
            else:
                if accessed:
                    patch_vary_headers(response, ('Cookie',))
                if (modified or settings.SESSION_SAVE_EVERY_REQUEST) and not empty:
                    if request.session.get_expire_at_browser_close():
                        max_age = None
                        expires = None
                    else:
                        max_age = request.session.get_expiry_age()
                        expires_time = time.time() + max_age
                        expires = http_date(expires_time)
                    # Save the session data and refresh the client cookie.
                    # Skip session save for 500 responses, refs #3881.
                    if response.status_code != 500:
                        try:
                            request.session.save()
                        except UpdateError:
                            raise SuspiciousOperation(
                                "The request's session was deleted before the "
                                "request completed. The user may have logged "
                                "out in a concurrent request, for example."
                            )
                        response.set_cookie(
                            settings.SESSION_COOKIE_NAME,
                            request.session.session_key, max_age=max_age,
                            expires=expires, domain=settings.SESSION_COOKIE_DOMAIN,
                            path=settings.SESSION_COOKIE_PATH,
                            secure=settings.SESSION_COOKIE_SECURE or None,
                            httponly=settings.SESSION_COOKIE_HTTPONLY or None,
                            samesite=settings.SESSION_COOKIE_SAMESITE,
                        )
        return response
```

#### Django Authentication

**Middleware**

加载uesr对象。

```py
class AuthenticationMiddleware(MiddlewareMixin):
    def process_request(self, request):
        assert hasattr(request, 'session'), (
            "The Django authentication middleware requires session middleware "
            "to be installed. Edit your MIDDLEWARE%s setting to insert "
            "'django.contrib.sessions.middleware.SessionMiddleware' before "
            "'django.contrib.auth.middleware.AuthenticationMiddleware'."
        ) % ("_CLASSES" if settings.MIDDLEWARE is None else "")
        request.user = SimpleLazyObject(lambda: get_user(request))
```

> 加载user对象，是延迟加载，只有当真正访问user对象，才会出发相关的动作。

```py
def get_user(request):
    if not hasattr(request, '_cached_user'):
        request._cached_user = auth.get_user(request)
    return request._cached_user


def get_user(request):
    """
    Return the user model instance associated with the given request session.
    If no user is retrieved, return an instance of `AnonymousUser`.
    """
    from .models import AnonymousUser
    user = None
    try:
        user_id = _get_user_session_key(request)
        backend_path = request.session[BACKEND_SESSION_KEY]
    except KeyError:
        pass
    else:
        if backend_path in settings.AUTHENTICATION_BACKENDS:
            backend = load_backend(backend_path)
            user = backend.get_user(user_id)
            # Verify the session
            if hasattr(user, 'get_session_auth_hash'):
                session_hash = request.session.get(HASH_SESSION_KEY)
                session_hash_verified = session_hash and constant_time_compare(
                    session_hash,
                    user.get_session_auth_hash()
                )
                if not session_hash_verified:
                    request.session.flush()
                    user = None

    return user or AnonymousUser()
```

> 如何是匿名用户，是`AnonymousUser()`

### Django Exception

*https://mengyangyang.org/2017/12/06/django-exception/*

基于django技术栈实现的WEB应用，生产环境中都会关闭DEBUG选项，Django默认只会输出很少的错误信息，不利于开发人员快速定位、解决问题。为了解决此问题，考虑定制Django默认的错误处理，还原错误现场，配合错误日志、邮件报警快速发现、解决BUG。

### Django Database Routing

django ORM数据模型配置数据库．

django支持多个数据库，通过django ORM定义数据模型，比如class User(Model)，无法通过class Meta配置管理该数据模型对应的数据库，只能使用默认数据库default．

django ConnectionRouter解决数据模型与数据库映射．

#### DB router

```python

import logging
from django.conf import settings

_logger = logging.getLogger('django')

class DatabaseRouter(object):
    """
    Database router to control the  models for differrent db.
    """

    DEFAULT_DB = 'default'

    def _db(self, model, **hints):
        db = getattr(model, '_database', None)
        if not db:
            return self.DEFAULT_DB

        if db in settings.DATABASES.keys():
            return db
        else:
            _logger.warn('%s not exist' % db)
            return self.DEFAULT_DB

    def db_for_read(self, model, **hints):
        return self._db(model, **hints)

    def db_for_write(self, model, **hints):
        return self._db(model, **hints)
    
```

#### Config DB router

In [27]:
DATABASE_ROUTERS = ['db_router.DatabaseRouter']

#### Theory

```python
class ConnectionRouter(object):

    @cached_property
    def routers(self):
        if self._routers is None:
            self._routers = settings.DATABASE_ROUTERS
        routers = []
        for r in self._routers:
            if isinstance(r, six.string_types):
                router = import_string(r)()
            else:
                router = r
            routers.append(router)
        return routers

    def _router_func(action):
        def _route_db(self, model, **hints):
            chosen_db = None
            for router in self.routers:
                try:
                    method = getattr(router, action)
                except AttributeError:
                    # If the router doesn't have a method, skip to the next one.
                    pass
                else:
                    chosen_db = method(model, **hints)
                    if chosen_db:
                        return chosen_db
            instance = hints.get('instance')
            if instance is not None and instance._state.db:
                return instance._state.db
            return DEFAULT_DB_ALIAS
        return _route_db

    db_for_read = _router_func('db_for_read')
    db_for_write = _router_func('db_for_write')
```

**Router initialization**

```python
router = ConnectionRouter()
```

**Call router**

```python
class QuerySet(object):

    @property
    def db(self):
        "Return the database that will be used if this query is executed now"
        if self._for_write:
            return self._db or router.db_for_write(self.model, **self._hints)
        return self._db or router.db_for_read(self.model, **self._hints)
    
```

## Django Rest Framework

*https://www.django-rest-framework.org/*